# Predicting Deutsche Bahn Train Delays  
## A Reproducible Baseline for Supervised Regression

**Objective:** Build a supervised regression model to predict train arrival delays (in minutes) for Deutsche Bahn trains using statistical learning methods.

**Target Variable:** `arrival_delay_m` - continuous variable representing delay in minutes

---

## 1. Environment Setup and Imports

### Google Colab Setup

In [ ]:
# Check if running in Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running in Google Colab")

    # Install required packages
    %pip install pandas numpy matplotlib seaborn scikit-learn scipy kagglehub -q

    # Mount Google Drive (optional - for saving results)
    from google.colab import drive
    drive.mount('/content/drive')

    # Set memory-efficient pandas options
    import pandas as pd
    pd.options.mode.chained_assignment = None
    pd.options.display.max_columns = 50

else:
    print("Running locally")

### Local Setup (Anaconda/Miniconda)

For local installation, follow these steps in your terminal:

```bash
# 1. Install Anaconda or Miniconda
# Download from: https://www.anaconda.com/download or https://docs.conda.io/en/latest/miniconda.html

# 2. Create a new conda environment
conda create -n ml-db-delays python=3.9 -y

# 3. Activate the environment
conda activate ml-db-delays

# 4. Install required packages
conda install -c conda-forge pandas numpy matplotlib seaborn scikit-learn scipy jupyter notebook ipykernel -y

# 5. Install additional packages via pip
pip install kagglehub

# 6. Add kernel to Jupyter
python -m ipykernel install --user --name ml-db-delays --display-name "ML DB Delays"

# 7. Launch Jupyter Notebook
jupyter notebook

# 8. Select the "ML DB Delays" kernel when creating/opening the notebook
```

### Mathematical Foundation
As per ITSL Chapter 2.1, we model the relationship between predictors and response as:

$$Y = f(X) + \epsilon$$

where:
- $Y$ is the response variable (arrival delay in minutes)
- $X = (X_1, X_2, ..., X_p)$ represents our $p$ predictors
- $f$ is the unknown systematic function we aim to estimate
- $\epsilon$ is random error with $E(\epsilon) = 0$ and $Var(\epsilon) = \sigma^2$

### Import Required Libraries

In [ ]:
# Standard library imports
import os
import sys
import gc
import warnings
from datetime import datetime
import psutil  # For memory monitoring

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
import sklearn
from sklearn.model_selection import (
    train_test_split, cross_val_score, GridSearchCV,
    KFold, learning_curve, validation_curve
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    make_scorer
)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.base import BaseEstimator, RegressorMixin

# Feature selection
try:
    from mlxtend.feature_selection import SequentialFeatureSelector as SFS
except:
    print("mlxtend not available - will use alternative feature selection")

# Statistical analysis
from scipy import stats

# Kaggle data loading
import kagglehub

# Configuration
warnings.filterwarnings('ignore')
np.random.seed(42)

# Memory-efficient pandas settings
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100

# Plotting configuration
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['figure.dpi'] = 100  # Reduce DPI for memory efficiency

# Memory monitoring function
def check_memory():
    """Monitor memory usage"""
    if 'psutil' in sys.modules:
        process = psutil.Process(os.getpid())
        mem_info = process.memory_info()
        return f"Memory Usage: {mem_info.rss / 1024 / 1024 / 1024:.2f} GB"
    else:
        return "psutil not available - install with: pip install psutil"

print("All packages imported successfully!")
print(check_memory())

---

## 2. Data Loading and Initial Inspection

### The Supervised Learning Workflow (Lecture Slides)

Following the supervised learning experiment steps:
1. **Training data** → **Preprocessing** → **Feature extraction** → **Feature selection** → **Training**
2. **Test data** → **Preprocessing** → **Selected feature extraction** → **Classifier** → **Classification result**


In [ ]:
# Download dataset
print("Downloading Deutsche Bahn delays dataset...")
path = kagglehub.dataset_download("nokkyu/deutsche-bahn-db-delays")
print(f"Dataset downloaded to: {path}")

# Find the CSV file
file_path = os.path.join(path, 'db_delays.csv')
print(f"\nLoading data from: {file_path}")

# Load data with optimized settings
print("\nLoading dataset...")
df = pd.read_csv(file_path, 
                 parse_dates=['arrival_plan', 'departure_plan', 'arrival_change', 'departure_change'],
                 low_memory=False)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
print(check_memory())

# For extremely large datasets, consider sampling
if len(df) > 2000000:  # If more than 2 million rows
    print(f"\nDataset has {len(df):,} rows. Sampling for manageable processing...")
    df = df.sample(n=min(1000000, len(df)), random_state=42)
    print(f"Working with {len(df):,} sampled rows")
    gc.collect()


### Initial Data Quality Assessment

As stated in the slides: "The data quality should be good" and "Before doing classification/regression experiments, you should be familiar with the data."


In [ ]:
# Basic information about the dataset
print("\n" + "="*80)
print("DATASET INFORMATION")
print("="*80)
print(df.info())

print("\n" + "="*80)
print("FIRST 5 ROWS")
print("="*80)
print(df.head())

print("\n" + "="*80)
print("TARGET VARIABLE STATISTICS (arrival_delay_m)")
print("="*80)
print(df['arrival_delay_m'].describe())

---

## 3. Data Preprocessing

### 3.1 Remove Duplicates

First step in data cleaning: remove duplicate records to ensure data quality.

In [ ]:
print("\n" + "="*80)
print("DUPLICATE REMOVAL")
print("="*80)

initial_rows = len(df)
df = df.drop_duplicates()
final_rows = len(df)

print(f"Rows before removing duplicates: {initial_rows:,}")
print(f"Rows after removing duplicates: {final_rows:,}")
print(f"Duplicates removed: {initial_rows - final_rows:,} ({(initial_rows - final_rows)/initial_rows*100:.2f}%)")

### 3.2 Missing Value Analysis and Treatment

Following best practices: analyze missing patterns before deciding on treatment strategy.

In [ ]:
print("\n" + "="*80)
print("MISSING VALUE ANALYSIS")
print("="*80)

# Calculate missing values
missing_stats = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100
})
missing_stats = missing_stats[missing_stats['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

print(missing_stats)

# Visualize missing patterns
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap of missing values
missing_cols = df.columns[df.isnull().any()].tolist()
if len(missing_cols) > 0:
    # Sample for visualization
    sample_size = min(1000, len(df))
    sample_df = df[missing_cols].sample(n=sample_size, random_state=42)
    
    sns.heatmap(sample_df.isnull(), cbar=True, ax=axes[0], cmap='viridis_r', 
                yticklabels=False)
    axes[0].set_title('Missing Values Pattern (Sample)')
    axes[0].set_xlabel('Features')

# Bar plot of missing percentages
missing_stats.plot(x='Column', y='Missing_Percentage', kind='barh', ax=axes[1], legend=False)
axes[1].set_xlabel('Missing Percentage (%)')
axes[1].set_title('Missing Values by Feature')

plt.tight_layout()
plt.show()

**Analysis:** The feature analysis reveals several important patterns:
1. Train category significantly impacts delays, with long-distance trains showing higher average delays
2. Clear temporal patterns exist, with peak hours showing increased delays
3. Strong correlation between departure and arrival delays suggests departure delay is a key predictor
4. Geographic variations indicate location-based features may improve predictions

---

## 4. Data Preprocessing

### Outlier Detection and Treatment
Following the slides on "Outliers, cross validation, resampling", we implement robust outlier detection.

#### Mathematical Framework for Outlier Detection
We use two approaches as discussed in the lecture:

1. **IQR Method**: Values outside $[Q_1 - 1.5 \times IQR, Q_3 + 1.5 \times IQR]$ are considered outliers
2. **Z-Score Method**: Values with $|z| > 3$ where $z = \frac{x - \mu}{\sigma}$ are considered outliers


In [ ]:
def detect_outliers(df, column, method='IQR'):
    """
    Detect outliers using IQR or Z-score method
    Reference: Slides on Outlier Detection
    """
    if method == 'IQR':
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = (df[column] < lower_bound) | (df[column] > upper_bound)
    elif method == 'Z-score':
        z_scores = np.abs(stats.zscore(df[column].dropna()))
        outliers = z_scores > 3

    return outliers, lower_bound if method == 'IQR' else -3, upper_bound if method == 'IQR' else 3

# Analyze outliers in target variable
outliers_iqr, lower_iqr, upper_iqr = detect_outliers(df, 'arrival_delay_m', method='IQR')
outliers_zscore, _, _ = detect_outliers(df, 'arrival_delay_m', method='Z-score')

print("Outlier Analysis:")
print("="*60)
print(f"Outliers detected by IQR method: {outliers_iqr.sum()} ({outliers_iqr.sum()/len(df)*100:.2f}%)")
print(f"Outliers detected by Z-score method: {outliers_zscore.sum()} ({outliers_zscore.sum()/len(df)*100:.2f}%)")
print(f"\nIQR bounds: [{lower_iqr:.2f}, {upper_iqr:.2f}] minutes")

# Visualize outlier impact
fig, axes = plt.subplots(1, 2, figsize=(14, 6), dpi=80)

# Original distribution
sample_viz = df.sample(min(50000, len(df)))
axes[0].hist(sample_viz['arrival_delay_m'].dropna(), bins=50, alpha=0.7, color='blue', edgecolor='black')
axes[0].axvline(upper_iqr, color='red', linestyle='--', label='IQR Upper Bound')
axes[0].set_xlabel('Arrival Delay (minutes)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Original Distribution with Outlier Threshold')
axes[0].set_xlim(-50, 200)
axes[0].legend()

# Distribution after outlier capping
df['arrival_delay_m_capped'] = df['arrival_delay_m'].clip(lower=lower_iqr, upper=upper_iqr)
axes[1].hist(sample_viz['arrival_delay_m'].clip(lower=lower_iqr, upper=upper_iqr),
             bins=50, alpha=0.7, color='green', edgecolor='black')
axes[1].set_xlabel('Arrival Delay (minutes)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution after Outlier Capping')
axes[1].set_xlim(-50, 200)

plt.tight_layout()
plt.show()

print(f"\nOutlier Treatment Decision: Cap delays at [{lower_iqr:.0f}, {upper_iqr:.0f}] minutes")

# Cleanup
del sample_viz, outliers_iqr, outliers_zscore
gc.collect()

### Missing Value Treatment Strategy

Based on the analysis:
1. Remove rows with missing target variable (arrival_delay_m)
2. Remove rows with missing critical features
3. Impute remaining missing values appropriately

In [ ]:
print("\n" + "="*80)
print("MISSING VALUE TREATMENT")
print("="*80)

# 1. Remove rows with missing target variable
initial_len = len(df)
df = df.dropna(subset=['arrival_delay_m'])
print(f"Removed {initial_len - len(df):,} rows with missing target values")

# 2. Remove rows with missing critical features
critical_features = ['eva_nr', 'category', 'departure_delay_m', 'long', 'lat']
before_critical = len(df)
df = df.dropna(subset=critical_features)
print(f"Removed {before_critical - len(df):,} rows with missing critical features")

# 3. Impute remaining missing values
# For categorical columns: fill with 'Unknown'
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna('Unknown')
        print(f"Filled {col} with 'Unknown'")

# For numerical columns: fill with median
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_cols:
    if col != 'arrival_delay_m' and df[col].isnull().any():
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)
        print(f"Filled {col} with median value: {median_val:.2f}")

print(f"\nFinal dataset size after missing value treatment: {len(df):,} rows")

### 3.3 Feature Engineering

Create domain-specific features based on the available data.

In [ ]:
print("\n" + "="*80)
print("FEATURE ENGINEERING")
print("="*80)

# Extract temporal features
df['hour'] = pd.to_datetime(df['departure_plan']).dt.hour
df['day_of_week'] = pd.to_datetime(df['departure_plan']).dt.dayofweek
df['month'] = pd.to_datetime(df['departure_plan']).dt.month
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_rush_hour'] = df['hour'].isin([7, 8, 9, 17, 18, 19]).astype(int)

# Station-based aggregated features
print("Creating station-based features...")
station_stats = df.groupby('station')['arrival_delay_m'].agg(['mean', 'std', 'count']).reset_index()
station_stats.columns = ['station', 'station_avg_delay', 'station_std_delay', 'station_traffic']
df = df.merge(station_stats, on='station', how='left')

# Category-based aggregated features
print("Creating category-based features...")
category_stats = df.groupby('category')['arrival_delay_m'].agg(['mean', 'std']).reset_index()
category_stats.columns = ['category', 'category_avg_delay', 'category_std_delay']
df = df.merge(category_stats, on='category', how='left')

print("\nNew features created:")
print("- Temporal: hour, day_of_week, month, is_weekend, is_rush_hour")
print("- Station-based: station_avg_delay, station_std_delay, station_traffic")
print("- Category-based: category_avg_delay, category_std_delay")

---

## 4. Exploratory Data Analysis

### 4.1 Target Variable Distribution

In [ ]:
print("\n" + "="*80)
print("TARGET VARIABLE ANALYSIS")
print("="*80)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Distribution plot
axes[0].hist(df['arrival_delay_m'], bins=100, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Arrival Delay (minutes)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Arrival Delays')
axes[0].set_xlim(-50, 200)

# Box plot
axes[1].boxplot(df['arrival_delay_m'], vert=True)
axes[1].set_ylabel('Arrival Delay (minutes)')
axes[1].set_title('Box Plot of Arrival Delays')

plt.tight_layout()
plt.show()

# Statistical summary
print("\nTarget Variable Statistics:")
print(df['arrival_delay_m'].describe())

### 4.2 Feature Correlations

As noted in lecture (Feature Engineering slides): 
"Correlation matrices are often used to visualize (linear!) dependency among features."

**Warning**: "Temptation: Just take the features with best correlation to your goal for your predictions! This is deeply wrong."


In [ ]:
# Select numerical features for correlation analysis
numerical_features = ['arrival_delay_m', 'departure_delay_m', 'category', 'eva_nr', 
                     'zip', 'long', 'lat', 'hour', 'day_of_week', 'month',
                     'is_weekend', 'is_rush_hour', 'station_avg_delay', 
                     'station_std_delay', 'category_avg_delay']

# Calculate correlation matrix
corr_matrix = df[numerical_features].corr()

# Visualize correlation matrix
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='coolwarm', center=0, square=True, linewidths=1,
            cbar_kws={"shrink": .8})
plt.title('Correlation Matrix of Numerical Features', fontsize=16)
plt.tight_layout()
plt.show()

# Top correlations with target
print("\n" + "="*80)
print("TOP CORRELATIONS WITH TARGET VARIABLE")
print("="*80)
target_corr = corr_matrix['arrival_delay_m'].sort_values(ascending=False)
print(target_corr[1:11])  # Exclude self-correlation

### 4.3 Feature Relationships


In [ ]:
# Analyze key relationships
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Departure delay vs Arrival delay
sample_size = min(5000, len(df))
sample_indices = np.random.choice(df.index, sample_size, replace=False)
axes[0, 0].scatter(df.loc[sample_indices, 'departure_delay_m'], 
                   df.loc[sample_indices, 'arrival_delay_m'], 
                   alpha=0.5, s=10)
axes[0, 0].set_xlabel('Departure Delay (minutes)')
axes[0, 0].set_ylabel('Arrival Delay (minutes)')
axes[0, 0].set_title('Departure Delay vs Arrival Delay')
axes[0, 0].plot([0, 100], [0, 100], 'r--', alpha=0.5)

# 2. Average delay by category
category_delays = df.groupby('category')['arrival_delay_m'].agg(['mean', 'std'])
category_delays.sort_values('mean').plot(y='mean', kind='barh', ax=axes[0, 1], 
                                        xerr='std', capsize=3)
axes[0, 1].set_xlabel('Average Delay (minutes)')
axes[0, 1].set_title('Average Delay by Train Category')

# 3. Hourly pattern
hourly_pattern = df.groupby('hour')['arrival_delay_m'].agg(['mean', 'std'])
axes[1, 0].errorbar(hourly_pattern.index, hourly_pattern['mean'], 
                    yerr=hourly_pattern['std'], marker='o', capsize=3)
axes[1, 0].set_xlabel('Hour of Day')
axes[1, 0].set_ylabel('Average Delay (minutes)')
axes[1, 0].set_title('Average Delay by Hour of Day')
axes[1, 0].grid(True, alpha=0.3)

# 4. Weekend effect
weekend_comparison = df.groupby('is_weekend')['arrival_delay_m'].apply(list)
axes[1, 1].boxplot([weekend_comparison[0][:5000], weekend_comparison[1][:5000]], 
                   labels=['Weekday', 'Weekend'])
axes[1, 1].set_ylabel('Arrival Delay (minutes)')
axes[1, 1].set_title('Delay Distribution: Weekday vs Weekend')

plt.tight_layout()
plt.show()

---

## 5. Data Splitting



### Train-Validation-Test Split (60-20-20)

Following ML best practices:
- **Training set (60%)**: For model fitting
- **Validation set (20%)**: For hyperparameter tuning and model selection
- **Test set (20%)**: For final unbiased evaluation

In [ ]:
print("\n" + "="*80)
print("DATA SPLITTING")
print("="*80)

# Define features and target
feature_columns = [
    # Numerical features
    'eva_nr', 'category', 'zip', 'long', 'lat', 'departure_delay_m',
    'hour', 'day_of_week', 'month', 'is_weekend', 'is_rush_hour',
    'station_avg_delay', 'station_std_delay', 'station_traffic',
    'category_avg_delay', 'category_std_delay',
    # Categorical features
    'station', 'state', 'line'
]

X = df[feature_columns].copy()
y = df['arrival_delay_m'].copy()

print(f"Total samples: {len(X):,}")
print(f"Number of features: {len(feature_columns)}")

# First split: separate test set (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

# Second split: separate train (60%) and validation (20%) from remaining 80%
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, shuffle=True  # 0.25 * 0.8 = 0.2
)

print(f"\nDataset splits:")
print(f"Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Validation set: {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Verify target distribution is similar across splits
print(f"\nTarget variable statistics by split:")
print(f"Train - Mean: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
print(f"Val   - Mean: {y_val.mean():.2f}, Std: {y_val.std():.2f}")
print(f"Test  - Mean: {y_test.mean():.2f}, Std: {y_test.std():.2f}")

---

## 6. Feature Preprocessing Pipeline

### StandardScaler and OneHotEncoder

From ISLR Section 4.7.4: "A good way to handle this problem is to **standardize** the data so that all variables are given a mean of zero and a standard deviation of one."

**Important**: Fit preprocessing only on training data to avoid data leakage!

In [ ]:
print("\n" + "="*80)
print("PREPROCESSING PIPELINE SETUP")
print("="*80)

# Identify numerical and categorical features
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

print(f"Numerical features ({len(numerical_features)}): {numerical_features}")
print(f"Categorical features ({len(categorical_features)}): {categorical_features}")

# Create preprocessing pipeline
# StandardScaler: transforms features to have mean=0 and std=1
# OneHotEncoder: creates binary features for each category
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), 
         categorical_features)
    ],
    remainder='drop'  # Drop any other columns
)

# IMPORTANT: Fit preprocessor on training data ONLY
print("\nFitting preprocessor on training data...")
preprocessor.fit(X_train)

# Get feature names after preprocessing
feature_names_after_preprocessing = (
    numerical_features + 
    [f"{cat}_{val}" for cat, vals in 
     zip(categorical_features, preprocessor.named_transformers_['cat'].categories_)
     for val in vals[1:]]  # drop='first' removes first category
)

print(f"\nTotal features after preprocessing: {len(feature_names_after_preprocessing)}")
print("Preprocessing pipeline created and fitted on training data only!")

**Analysis:** The cross-validation results show consistent performance across folds, indicating stable model estimates. The tree-based model shows lower bias but higher variance compared to linear models, consistent with the bias-variance tradeoff discussed in ITSL Section 2.2.2.

### Learning Curves Analysis
Following ITSL concepts on model complexity and sample size effects.


In [ ]:
# Plot learning curves for best two models only
def plot_learning_curves(estimator, title, X, y, cv=3,
                        train_sizes=np.linspace(0.2, 1.0, 5)):
    """Memory-efficient learning curves"""

    # Use at most 50k samples
    if len(X) > 50000:
        indices = np.random.choice(len(X), 50000, replace=False)
        X_sample = X.iloc[indices]
        y_sample = y.iloc[indices]
    else:
        X_sample = X
        y_sample = y

    train_sizes, train_scores, val_scores = learning_curve(
        estimator, X_sample, y_sample, cv=cv, n_jobs=-1,
        train_sizes=train_sizes,
        scoring='neg_mean_absolute_error'
    )

    return train_sizes, -train_scores, -val_scores

# Plot for Linear Regression and Random Forest
fig, axes = plt.subplots(1, 2, figsize=(14, 6), dpi=80)

models_to_plot = [
    ('Linear Regression', lr_pipeline),
    ('Random Forest', rf_grid_search.best_estimator_)
]

for idx, (name, model) in enumerate(models_to_plot):
    print(f"Computing learning curves for {name}...")
    train_sizes, train_scores, val_scores = plot_learning_curves(
        model, name, X_train, y_train
    )

    ax = axes[idx]
    train_mean = train_scores.mean(axis=1)
    train_std = train_scores.std(axis=1)
    val_mean = val_scores.mean(axis=1)
    val_std = val_scores.std(axis=1)

    ax.fill_between(train_sizes, train_mean - train_std,
                    train_mean + train_std, alpha=0.1, color="r")
    ax.fill_between(train_sizes, val_mean - val_std,
                    val_mean + val_std, alpha=0.1, color="g")
    ax.plot(train_sizes, train_mean, 'o-', color="r", label="Training score")
    ax.plot(train_sizes, val_mean, 'o-', color="g", label="Validation score")

    ax.set_xlabel("Training Set Size")
    ax.set_ylabel("MAE (minutes)")
    ax.set_title(f"Learning Curves - {name}")
    ax.legend(loc="best")
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 7. Feature Selection

### Forward Stepwise Selection

From ISLR Chapter 6.1.2:

**Algorithm 6.2 Forward stepwise selection**
1. Let $M_0$ denote the null model, which contains no predictors
2. For $k = 0, ..., p-1$:
   - Consider all $p-k$ models that augment the predictors in $M_k$ with one additional predictor
   - Choose the best among these $p-k$ models, and call it $M_{k+1}$
3. Select a single best model from among $M_0, ..., M_p$ using cross-validation

In [ ]:
print("\n" + "="*80)
print("FEATURE SELECTION")
print("="*80)

# Transform data using fitted preprocessor
X_train_transformed = preprocessor.transform(X_train)
X_val_transformed = preprocessor.transform(X_val)

print(f"Transformed training data shape: {X_train_transformed.shape}")

# Step 1: Use F-score for initial feature ranking
# F-statistic measures the linear dependency between each feature and target
selector = SelectKBest(score_func=f_regression, k='all')
selector.fit(X_train_transformed, y_train)

# Get feature scores
feature_scores = pd.DataFrame({
    'feature': feature_names_after_preprocessing,
    'f_score': selector.scores_
}).sort_values('f_score', ascending=False)

# Visualize top features
plt.figure(figsize=(10, 8))
top_n = 20
feature_scores.head(top_n).plot(x='feature', y='f_score', kind='barh')
plt.xlabel('F-score')
plt.title(f'Top {top_n} Features by F-score')
plt.tight_layout()
plt.show()

print(f"\nTop 15 features by F-score:")
print(feature_scores.head(15))

# Step 2: Perform forward selection on top features
# Due to computational constraints, we'll select from top 50 features
n_top_features = min(50, X_train_transformed.shape[1])
top_indices = feature_scores.head(n_top_features).index.tolist()

# Get indices of top features
top_feature_mask = np.zeros(X_train_transformed.shape[1], dtype=bool)
for idx in top_indices:
    top_feature_mask[idx] = True

X_train_top = X_train_transformed[:, top_feature_mask]
X_val_top = X_val_transformed[:, top_feature_mask]

print(f"\nPerforming forward selection on top {n_top_features} features...")

# Simplified forward selection using RFE (Recursive Feature Elimination)
# RFE is computationally more efficient than full forward selection
lr_selector = LinearRegression()
n_features_to_select = 20  # Final number of features

rfe = RFE(estimator=lr_selector, n_features_to_select=n_features_to_select, step=1)
rfe.fit(X_train_top, y_train)

# Get selected features
selected_feature_indices = np.where(rfe.support_)[0]
selected_features = [feature_scores.iloc[i]['feature'] for i in selected_feature_indices]

print(f"\nForward selection chose {len(selected_features)} features:")
for i, feat in enumerate(selected_features, 1):
    print(f"{i:2d}. {feat}")

# Create final selector for pipeline
final_selector = SelectKBest(score_func=f_regression, k=30)  # Select top 30 features

---

## 8. Model Development

### 8.1 Linear Regression (Base Model)

The linear regression model (ISLR Chapter 3):

$$Y = \beta_0 + \beta_1X_1 + \beta_2X_2 + ... + \beta_pX_p + \epsilon$$

We find $\hat{\beta}$ by minimizing the residual sum of squares (RSS):

$$RSS = \sum_{i=1}^{n}(y_i - \hat{y}_i)^2 = \sum_{i=1}^{n}(y_i - \beta_0 - \sum_{j=1}^{p}\beta_jx_{ij})^2$$

In [ ]:
print("\n" + "="*80)
print("MODEL DEVELOPMENT")
print("="*80)

# Evaluation function
def evaluate_model(y_true, y_pred, model_name, dataset_name):
    """Calculate and display model performance metrics"""
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{model_name} - {dataset_name} Performance:")
    print(f"  MAE:  {mae:.2f} minutes")
    print(f"  RMSE: {rmse:.2f} minutes")
    print(f"  R²:   {r2:.4f}")
    
    return {'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2}

# Create Linear Regression pipeline
print("\n1. LINEAR REGRESSION (Base Model)")
print("-"*60)

lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', final_selector),
    ('regressor', LinearRegression())
])

# Fit the model
print("Training Linear Regression model...")
lr_pipeline.fit(X_train, y_train)

# Make predictions
y_train_pred_lr = lr_pipeline.predict(X_train)
y_val_pred_lr = lr_pipeline.predict(X_val)

# Evaluate
train_metrics_lr = evaluate_model(y_train, y_train_pred_lr, "Linear Regression", "Training")
val_metrics_lr = evaluate_model(y_val, y_val_pred_lr, "Linear Regression", "Validation")

### 8.2 Residual Analysis for Outlier Detection

From lecture slides (Outliers, cross validation, resampling):
- "Plot the residuals (regressions)"
- "Outliers: Plot the residuals"
- "The outlier has a studentized residual of 6; typically we expect values between −3 and 3"

In [ ]:
print("\n" + "="*80)
print("RESIDUAL ANALYSIS FOR OUTLIER DETECTION")
print("="*80)

# Calculate residuals
residuals_train = y_train - y_train_pred_lr
residuals_val = y_val - y_val_pred_lr

# Create comprehensive residual plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Residuals vs Fitted Values
axes[0, 0].scatter(y_train_pred_lr, residuals_train, alpha=0.5, s=10)
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Fitted Values')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Residuals vs Fitted Values')

# Add smoothed line to detect patterns
from scipy.signal import savgol_filter
sorted_indices = np.argsort(y_train_pred_lr)
sorted_fitted = y_train_pred_lr[sorted_indices]
sorted_residuals = residuals_train.iloc[sorted_indices]
if len(sorted_fitted) > 50:
    window_size = min(51, len(sorted_fitted) // 10)
    if window_size % 2 == 0:
        window_size += 1
    smoothed = savgol_filter(sorted_residuals, window_size, 3)
    axes[0, 0].plot(sorted_fitted, smoothed, 'g-', linewidth=2, label='Smoothed trend')
    axes[0, 0].legend()

# 2. Q-Q plot for normality check
stats.probplot(residuals_train, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('Normal Q-Q Plot')
axes[0, 1].get_lines()[1].set_color('red')

# 3. Histogram of residuals
axes[0, 2].hist(residuals_train, bins=50, edgecolor='black', alpha=0.7, density=True)
axes[0, 2].set_xlabel('Residuals')
axes[0, 2].set_ylabel('Density')
axes[0, 2].set_title('Distribution of Residuals')

# Add normal distribution overlay
residual_std = np.std(residuals_train)
residual_mean = np.mean(residuals_train)
x_norm = np.linspace(residual_mean - 4*residual_std, residual_mean + 4*residual_std, 100)
axes

---

## References

1. James, G., Witten, D., Hastie, T., Tibshirani, R., & Taylor, J. (2023). *An Introduction to Statistical Learning with Applications in Python* (ISLP). Springer.

2. Mayer, M. (2025). *Machine Learning Course Materials*. TH Deggendorf.

3. Scikit-learn Documentation. https://scikit-learn.org/

4. Deutsche Bahn Delays Dataset. Kaggle. https://www.kaggle.com/datasets/nokkyu/deutsche-bahn-db-delays


---

<!-- **Team Contributions:**

**Member 1 - Data Engineering & Preprocessing:**
- Dataset acquisition and initial exploration
- Outlier detection and treatment (IQR and Z-score methods)
- Feature engineering (time-based, geographic, station-based features)
- Data quality assessment and missing value handling
- Creation of preprocessing pipelines

**Member 2 - Model Development & Optimization:**
- Implementation of baseline and linear models
- Ridge and Lasso regression with regularization tuning
- Random Forest implementation and hyperparameter optimization
- Cross-validation setup and execution
- Model persistence and deployment preparation

**Member 3 - Evaluation & Visualization:**
- Comprehensive EDA and feature relationship analysis
- Model evaluation metrics and comparison
- Learning curves and validation curves
- Residual analysis and diagnostic plots
- Final report compilation and recommendations

**Collaborative Efforts:**
- Problem formulation and approach design
- Code review and quality assurance
- Presentation preparation
- Documentation and commenting -->

---